## Procesamiento

In [286]:
import funciones_comunes
import funciones_mongo 
import funciones_neo4j 
import importlib
importlib.reload(funciones_comunes) # recarga el módulo después de modificarl
importlib.reload(funciones_mongo) # recarga el módulo después de modificarl
importlib.reload(funciones_neo4j)


<module 'funciones_neo4j' from '/work/funciones_neo4j.py'>

In [257]:
# info = {"usuarios" : [], "destinos" : [], "hoteles": [], "actividades":[], "reservas":[] }
info = funciones_comunes.procesar_datos()
print(info)


{'usuarios': [{'usuario_id': 1, 'nombre': 'María Pérez', 'email': 'maria.perez@example.com', 'telefono': '+54 11 4567 1234'}, {'usuario_id': 2, 'nombre': 'Juan López', 'email': 'juan.lopez@example.com', 'telefono': '+54 221 334 5566'}, {'usuario_id': 3, 'nombre': 'Carla Gómez', 'email': 'carla.gomez@example.com', 'telefono': '+54 261 789 2233'}, {'usuario_id': 4, 'nombre': 'Luis Fernández', 'email': 'luis.fernandez@example.com', 'telefono': '+54 299 444 9988'}, {'usuario_id': 5, 'nombre': 'Ana Torres', 'email': 'ana.torres@example.com', 'telefono': '+54 381 123 4567'}], 'destinos': [{'destino_id': 1, 'ciudad': 'Bariloche', 'pais': 'Argentina', 'tipo': 'Montaña', 'precio_promedio': 90000}, {'destino_id': 2, 'ciudad': 'Cancún', 'pais': 'México', 'tipo': 'Playa', 'precio_promedio': 150000}, {'destino_id': 3, 'ciudad': 'Madrid', 'pais': 'España', 'tipo': 'Cultural', 'precio_promedio': 110000}, {'destino_id': 4, 'ciudad': 'Roma', 'pais': 'Italia', 'tipo': 'Histórico', 'precio_promedio': 100

## | MongoDB: |

In [279]:
funciones_mongo.insertar_varios_documentos("usuarios",info["usuarios"])

In [280]:
funciones_mongo.insertar_varios_documentos("destinos",info["destinos"]) 

In [281]:
funciones_mongo.insertar_varios_documentos("hoteles",info["hoteles"]) 

In [282]:
funciones_mongo.insertar_varios_documentos("actividades",info["actividades"]) 

In [283]:
funciones_mongo.insertar_varios_documentos("reservas",info["reservas"]) 

In [284]:
for elem in funciones_mongo.db.hoteles.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.destinos.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.usuarios.find():
    print(elem)
print('')
for elem in funciones_mongo.db.actividades.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.reservas.find():
    print(elem) 

{'_id': ObjectId('68f54d3691d915677d6f39db'), 'hotel_id': 1, 'nombre': 'Hotel Sol', 'ciudad': 'Bariloche', 'precio': 85000, 'calificacion': 4, 'servicios': ['wifi', 'pileta', 'desayuno']}
{'_id': ObjectId('68f54d3691d915677d6f39dc'), 'hotel_id': 2, 'nombre': 'Cumbres Andinas', 'ciudad': 'Bariloche', 'precio': 120000, 'calificacion': 5, 'servicios': ['wifi', 'spa', 'pileta']}
{'_id': ObjectId('68f54d3691d915677d6f39dd'), 'hotel_id': 3, 'nombre': 'Altos del Norte', 'ciudad': 'Jujuy', 'precio': 60000, 'calificacion': 3, 'servicios': ['wifi']}
{'_id': ObjectId('68f54d3691d915677d6f39de'), 'hotel_id': 4, 'nombre': 'Montaña Real', 'ciudad': 'Mendoza', 'precio': 95000, 'calificacion': 4, 'servicios': ['wifi', 'pileta']}
{'_id': ObjectId('68f54d3691d915677d6f39df'), 'hotel_id': 5, 'nombre': 'Estancia Colonial', 'ciudad': 'Córdoba', 'precio': 70000, 'calificacion': 4, 'servicios': ['wifi', 'desayuno']}

{'_id': ObjectId('68f54d3591d915677d6f39d6'), 'destino_id': 1, 'ciudad': 'Bariloche', 'pais'

Base de datos:

viajes ← (esa es la variable db) 
Colecciones dentro de viajes:

usuarios

destinos

hoteles

actividades

reservas

Ejemplo visual:
viajes
 ├── usuarios
 │    ├── { "usuario_id": 1, "nombre": "Juan", "email": "juan@mail.com" }
 │    └── { "usuario_id": 2, "nombre": "Ana", "email": "ana@mail.com" }
 ├── destinos
 │    ├── { "destino_id": 1, "nombre": "Roma" }
 │    └── { "destino_id": 2, "nombre": "Tokio" }
 ├── hoteles
 │    └── { "hotel_id": 1, "nombre": "Hilton", "destino_id": 1 }
 ├── actividades
 │    └── { "actividad_id": 1, "nombre": "Tour guiado", "destino_id": 1 }
 └── reservas
      └── { "reserva_id": 1, "usuario_id": 1, "hotel_id": 1, "fecha": "2025-10-18" }

In [275]:
def drop_dataset():
    lista = ["usuarios","destinos","hoteles","actividades","reservas"]
    for elem in lista:
        db[elem].drop() # Elimina por completo la colección destinos dentro de la base de datos viajes.

In [276]:
drop_dataset()

## | Neo4J: |

In [290]:
from funciones_neo4j import insertar_varios_nodos,insertar_varias_relaciones
from funciones_comunes import obtener_datos_mongo

In [291]:
# Cargar nodos directamente desde Mongo
usuarios = obtener_datos_mongo("usuarios")
destinos = obtener_datos_mongo("destinos")

insertar_varios_nodos("Usuario", usuarios)
insertar_varios_nodos("Destino", destinos)

# Cargar relaciones VISITO desde la colección 'reservas'
reservas = obtener_datos_mongo("reservas")

rel_visitas = [
    {
        "label_origen": "Usuario",
        "prop_origen": "usuario_id",
        "valor_origen": r["usuario_id"],
        "label_destino": "Destino",
        "prop_destino": "destino_id",
        "valor_destino": r["destino_id"],
        "tipo": "VISITO"
    }
    for r in reservas
]

insertar_varias_relaciones(rel_visitas)

In [294]:
with driver.session() as session:
    # Contar relaciones VISITO
    resultado = session.execute_read(
        lambda tx: tx.run(
            "MATCH (:Usuario)-[r:VISITO]->(:Destino) RETURN count(r) AS total"
        ).single()
    )
    print("Total relaciones VISITO:", resultado["total"])

    # Opcional: mostrar algunas relaciones
    relaciones = session.execute_read(
        lambda tx: tx.run(
            "MATCH (u:Usuario)-[r:VISITO]->(d:Destino) RETURN u.usuario_id, d.destino_id LIMIT 10"
        ).data()
    )
    print(relaciones)

Total relaciones VISITO: 5
[{'u.usuario_id': 1, 'd.destino_id': 2}, {'u.usuario_id': 2, 'd.destino_id': 1}, {'u.usuario_id': 3, 'd.destino_id': 3}, {'u.usuario_id': 1, 'd.destino_id': 4}, {'u.usuario_id': 5, 'd.destino_id': 5}]


## | Redis :| 